<a href="https://colab.research.google.com/github/Ddkaba/AutoLogReg/blob/main/AutoLogReg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os
import patoolib
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
# Укажите имя вашего RAR-архива
rar_file = 'complaints.rar'  # замените на имя вашего файла

# Распакуйте архив в папку 'extracted_files'
patoolib.extract_archive(rar_file, outdir='extracted_files')

# Укажите имя CSV-файла, который вы хотите прочитать
csv_file = 'extracted_files/complaints.csv'  # замените на имя вашего CSV-файла

# Прочитайте CSV-файл
cars_df = pd.read_csv(csv_file)

# Посмотрите первые 5 строк
print(cars_df.head())

INFO patool: Extracting complaints.rar ...
INFO:patool:Extracting complaints.rar ...
INFO patool: running /usr/bin/unrar x -kb -or -- /content/complaints.rar
INFO:patool:running /usr/bin/unrar x -kb -or -- /content/complaints.rar
INFO patool: ... complaints.rar extracted to `extracted_files'.
INFO:patool:... complaints.rar extracted to `extracted_files'.


   odiNumber                manufacturer  crash   fire  numberOfInjuries  \
0   10702705           Kia America, Inc.   True  False                 0   
1   10713088           Kia America, Inc.  False  False                 0   
2   10713503    ALUMA TOWER COMPANY, INC  False   True                 0   
3   10715078           Kia America, Inc.  False  False                 0   
4   10715844  Volvo Trucks North America  False  False                 0   

   numberOfDeaths dateOfIncident dateComplaintFiled          vin  \
0               0     03/24/2015         03/30/2015  5XYPG4A36GG   
1               0     04/24/2015         04/27/2015  5XYPH4A15GG   
2               0     04/28/2015         04/29/2015  1YGAE1629GB   
3               0     04/30/2015         05/07/2015          NaN   
4               0     03/06/2015         05/12/2015  4V4NC9EH7GN   

          components                                            summary  \
0   SEAT BELTS,SEATS        SEAT BELT DID NOT UNLATCH AFTER

In [3]:
print("Общая информация")
print(cars_df.info())

print(f"Количество записей (объектов): {cars_df.shape[0]}")
print(f"Количество признаков (фич): {cars_df.shape[1]}")

print("\nНазвания столбцов:")
print(cars_df.columns.tolist())

print("\nТипы данных:")
print(cars_df.dtypes)

print("\nПропущенные значения:")
missing_values = cars_df.isnull().sum()
print(missing_values)
print(f"Общее количество пропущенных значений: {missing_values.sum()}")

print("Целевая переменная")
target_column = 'components'
if target_column in cars_df.columns:
    print(f"\nЦелевая переменная: {target_column}")
    print(f"Тип данных целевой переменной: {cars_df[target_column].dtype}")
    unique_values = cars_df[target_column].unique()
    print(f"Уникальные значения целевой переменной (первые 20): {unique_values[:20]}")
    print(f"Всего уникальных значений: {unique_values.size}")
    if cars_df[target_column].nunique() <= 20:
        print("Распределение классов:")
        print(cars_df[target_column].value_counts())
        print("Процентное соотношение классов:")
        print(cars_df[target_column].value_counts(normalize=True) * 100)

print("Статистика")
print(cars_df.describe())

print("Анализ кат. признаков")
categorical_features = []
for col in cars_df.columns:
    unique_values = cars_df[col].nunique(dropna=True)
    if unique_values <= 10:
        categorical_features.append(col)
        print(f"{col}: {unique_values} уникальных значений - {cars_df[col].unique()}")

print(f"\nВсего категориальных признаков: {len(categorical_features)}")


Общая информация
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233820 entries, 0 to 233819
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   odiNumber           233820 non-null  int64 
 1   manufacturer        233802 non-null  object
 2   crash               233820 non-null  bool  
 3   fire                233820 non-null  bool  
 4   numberOfInjuries    233820 non-null  int64 
 5   numberOfDeaths      233820 non-null  int64 
 6   dateOfIncident      233820 non-null  object
 7   dateComplaintFiled  233820 non-null  object
 8   vin                 228677 non-null  object
 9   components          233810 non-null  object
 10  summary             233809 non-null  object
 11  products            233820 non-null  object
 12  make                233820 non-null  object
 13  model               233820 non-null  object
 14  modelYear           233820 non-null  int64 
dtypes: bool(2), int64(4), object(9)
me

In [4]:
# Удаление нерелевантных признаков
cols_to_drop = ['odiNumber', 'manufacturer', 'vin', 'products', 'dateOfIncident','dateComplaintFiled']

# Выбираем рабочий датафрейм: если уже есть сгенерированный (cars_df), используем его,
# иначе — исходный cars_df.
df_ref = cars_df if 'dataset' in globals() else cars_df

print('До:', df_ref.shape)
# Удаляем только те признаки, которые реально присутствуют в датафрейме
df_ref.drop(columns=[c for c in cols_to_drop if c in df_ref.columns], axis=1, inplace=True)
print('После:', df_ref.shape)
print('Текущие столбцы:')
print(df_ref.columns.tolist())

# Если мы работаем с исходным dataset (dataset отсутствует),
# обновляем его, чтобы изменения сохранились.
if 'cars_df' not in globals():
    cars_df = df_ref

До: (233820, 15)
После: (233820, 9)
Текущие столбцы:
['crash', 'fire', 'numberOfInjuries', 'numberOfDeaths', 'components', 'summary', 'make', 'model', 'modelYear']


In [11]:
cars_df = cars_df.dropna(subset=['components'])
print(cars_df.shape)
cars_df = cars_df.copy()
cars_df['num_components'] = cars_df['components'].apply(lambda x: len(x.split(',')) if isinstance(x, str) else 0)
print(cars_df.shape)
print('Текущие столбцы:')
print(cars_df.columns.tolist())

(233810, 10)
(233810, 10)
Текущие столбцы:
['crash', 'fire', 'numberOfInjuries', 'numberOfDeaths', 'components', 'summary', 'make', 'model', 'modelYear', 'num_components']


In [12]:
def standardize_features(dataset):
    """
    Стандартизирует числовые признаки в датасете (z = (x - mean) / std).

    Параметры:
        dataset (pd.DataFrame): Входной датасет.

    Возвращает:
        pd.DataFrame: Стандартизированный датасет.
    """
    # Создаем копию, чтобы не изменять оригинал
    base_df = dataset.copy()

    # Отбираем числовые признаки
    feature_cols = base_df.select_dtypes(include=[np.number]).columns.tolist()

    # Проверка, есть ли числовые признаки
    if not feature_cols:
        raise ValueError("В датасете нет числовых признаков для стандартизации.")

    # Стандартизация
    scaler = StandardScaler()
    scaled = scaler.fit_transform(base_df[feature_cols])

    # Создаем DataFrame с стандартизированными признаками
    standardized_df = pd.DataFrame(scaled, columns=feature_cols, index=base_df.index)

    # Возвращаем все столбцы, заменяя числовые на стандартизированные
    base_df[feature_cols] = standardized_df[feature_cols]

    # Вывод информации
    print('Стандартизированы признаки:', feature_cols)
    print('Форма:', standardized_df.shape)

    # Проверка средних и стандартных отклонений
    means = base_df[feature_cols].mean().round(4)
    stdevs = base_df[feature_cols].std(ddof=0).round(4)
    print('\nСредние по столбцам (ожид. ≈ 0):')
    print(means)
    print('\nСт. отклонения (ожид. ≈ 1):')
    print(stdevs)
    print('\n')

    return base_df

# Пример использования
cars_df_standardized = standardize_features(cars_df)

print("\nТипы данных:")
print(cars_df_standardized.dtypes)


Стандартизированы признаки: ['numberOfInjuries', 'numberOfDeaths', 'modelYear', 'num_components']
Форма: (233810, 4)

Средние по столбцам (ожид. ≈ 0):
numberOfInjuries    0.0
numberOfDeaths      0.0
modelYear          -0.0
num_components     -0.0
dtype: float64

Ст. отклонения (ожид. ≈ 1):
numberOfInjuries    1.0
numberOfDeaths      1.0
modelYear           1.0
num_components      1.0
dtype: float64



Типы данных:
crash                  bool
fire                   bool
numberOfInjuries    float64
numberOfDeaths      float64
components           object
summary              object
make                 object
model                object
modelYear           float64
num_components      float64
dtype: object


In [9]:
# Предположим, что ваш DataFrame называется cars_df
# Выберите целевую переменную (например, 'components' или бинарный признак)
TARGET = 'components'  # или другой столбец, который вы хотите предсказывать

# Разделение на признаки и целевую переменную
X = cars_df.drop(columns=[TARGET])
y = cars_df[TARGET]

# Размер тестовой выборки (20%)
test_size = 0.2
# Размер валидационной выборки (25% от оставшихся 80%, что даст 20% от всего датасета)
val_size = 0.25
seed = 42  # для воспроизводимости результатов

# 1) Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=seed
)

# 2) Разделение обучающей выборки на обучающую и валидационную
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=val_size, random_state=seed
)

# Вывод форм выборок
print('Shapes:')
print('X_train:', X_train.shape, 'X_val:', X_val.shape, 'X_test:', X_test.shape)
print('y_train:', y_train.shape, 'y_val:', y_val.shape, 'y_test:', y_test.shape)


Shapes:
X_train: (140286, 9) X_val: (46762, 9) X_test: (46762, 9)
y_train: (140286,) y_val: (46762,) y_test: (46762,)
